<a href="https://colab.research.google.com/github/qasimzee/isic2024/blob/main/isic_2024_challenge_table_voting_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install scikit-learn==1.2.2
!pip install numpy==1.26.4

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb


In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
!ls "/content/drive/MyDrive/kaggle/isic-2024-data"

cnn.pkl			 sample_submission.csv	test-metadata.csv	  train-metadata.csv
logistic_regression.pkl  scaler.pkl		test-metadata.csv.gsheet  train-metadata-v1.csv
random_forest_model.pkl  test-image.hdf5	train-image.hdf5


In [26]:
METADATA_FILE = "/content/drive/MyDrive/kaggle/isic-2024-data/train-metadata.csv"
metadata_df = pd.read_csv(METADATA_FILE)


<ipython-input-26-6d834c4c6865>:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_df = pd.read_csv(METADATA_FILE)


In [27]:
train_df, test_df = train_test_split(metadata_df, test_size=0.2, random_state=42)

Y_test = test_df['target']


In [28]:
Y = train_df["target"]

dropped_columns = ["isic_id", "target", "patient_id", "lesion_id", "iddx_full", "iddx_1", "iddx_2", "iddx_3", "iddx_4", "iddx_5", "mel_mitotic_index", "mel_thick_mm", "tbp_lv_dnn_lesion_confidence"]
X = train_df.drop(columns=dropped_columns)

# Replace NaN values
X['sex'] = X['sex'].replace(pd.NA, 'unknown')
X['anatom_site_general'] = X['anatom_site_general'].replace(pd.NA, 'unknown')

X.fillna(0, inplace=True)

In [29]:
# Identify columns with mixed types

for column in X.columns:
  if X[column].apply(type).nunique() > 1:
    print(f"{column}: {X[column].apply(type).unique()}")


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Identify categorical features
categorical_columns = X.select_dtypes(include=['object']).columns
numerical_columns = X.select_dtypes(include=['number']).columns

# Create a ColumnTransformer with OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', drop='first'), categorical_columns),  # One-hot encode categorical columns
        ('num', StandardScaler(), numerical_columns)  # Scale numerical columns
    ]
)

X_train_encoded = preprocessor.fit_transform(X)


# Assuming you've already preprocessed your categorical variables
model = Sequential([
    Dense(64, input_shape=(X_train_encoded.shape[1],), activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
class_weight = {0: 1, 1: 100}  # Adjust the weight ratio to balance the classes

model.fit(X_train_encoded, Y, epochs=20, batch_size=32)



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 32s 3ms/step - accuracy: 0.9941 - loss: 0.0182
Epoch 2/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 38s 3ms/step - accuracy: 0.9991 - loss: 0.0059
Epoch 3/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 54s 4ms/step - accuracy: 0.9991 - loss: 0.0051
Epoch 4/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 27s 3ms/step - accuracy: 0.9991 - loss: 0.0053
Epoch 5/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 28s 3ms/step - accuracy: 0.9990 - loss: 0.0055
Epoch 6/20
 1975/10027 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.9989 - loss: 0.0054

In [ ]:
# test_df['sex'] = test_df['sex'].replace(pd.NA, 'unknown')
# test_df['anatom_site_general'] = test_df['anatom_site_general'].replace(pd.NA, 'unknown')
# test_df['lesion_id'] = test_df['lesion_id'].replace(pd.NA, 'unknown')
# test_df['iddx_2'] = test_df['iddx_2'].replace(pd.NA, 'unknown')
# test_df['iddx_3'] = test_df['iddx_3'].replace(pd.NA, 'unknown')
# test_df['iddx_4'] = test_df['iddx_4'].replace(pd.NA, 'unknown')
# test_df['mel_mitotic_index'] = test_df['mel_mitotic_index'].replace(pd.NA, 'unknown')

# test_df.fillna(0, inplace=True)
# # Identify columns with mixed types
# for column in test_df.columns:
#   if test_df[column].apply(type).nunique() > 1:
#     print(f"{column}: {test_df[column].unique()}")


test_isic_ids = test_df['isic_id']
test_df = test_df.drop(columns=dropped_columns)
test_df.head()

In [ ]:
# Make predictions on the test data
X_test_encoded = preprocessor.fit_transform(test_df)
y_pred_proba = model.predict(X_test_encoded)  # Probability of positive class (e.g., malignant)

# Create a DataFrame to map predictions back to 'isic_id'
predictions_df = pd.DataFrame({
    'isic_id': test_isic_ids,
    'target': y_pred_proba.flatten()
})


predictions_df


In [33]:
y_pred_proba = np.nan_to_num(y_pred_proba, nan=0)


In [34]:
y_pred_proba.max()

0.9221462

In [35]:
Y_test

,target
278442,0
215021,0
209685,0
29648,0
323386,0
...,...
2741,0
239797,0
243397,0
86684,0


In [ ]:
import numpy as np
from sklearn.metrics import roc_curve, auc


# Compute the ROC curve
fpr, tpr, thresholds = roc_curve(Y_test, y_pred_proba)

# Specify the range of TPR you are interested in (e.g., above 80%)
# Filter the points where TPR is above 0.8
tpr_threshold = 0.8
valid_range = tpr >= tpr_threshold

# Compute pAUC within the valid range
partial_auc = auc(fpr[valid_range], tpr[valid_range])

print(f"Partial AUC for TPR > 80%: {partial_auc:.2f}")


In [37]:
import pickle
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

# Assume you have trained your pipeline as described before

# Save the entire pipeline to a pickle file
with open('/content/drive/MyDrive/kaggle/isic-2024-data/sequential_dnn.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Model pipeline saved successfully!")


Model pipeline saved successfully!
